In [201]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msn

from datetime import date

from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [202]:
# data import
traindata = pd.read_csv('train.csv')
testdata = pd.read_csv('test.csv')

In [203]:
traindata.head(5)

Id   Open Date        City  City Group Type  P1   P2   P3   P4  P5  ...  \
0   0  07/17/1999    İstanbul  Big Cities   IL   4  5.0  4.0  4.0   2  ...   
1   1  02/14/2008      Ankara  Big Cities   FC   4  5.0  4.0  4.0   1  ...   
2   2  03/09/2013  Diyarbakır       Other   IL   2  4.0  2.0  5.0   2  ...   
3   3  02/02/2012       Tokat       Other   IL   6  4.5  6.0  6.0   4  ...   
4   4  05/09/2009   Gaziantep       Other   IL   3  4.0  3.0  4.0   2  ...   

   P29  P30  P31  P32  P33  P34  P35  P36  P37    revenue  
0  3.0    5    3    4    5    5    4    3    4  5653753.0  
1  3.0    0    0    0    0    0    0    0    0  6923131.0  
2  3.0    0    0    0    0    0    0    0    0  2055379.0  
3  7.5   25   12   10    6   18   12   12    6  2675511.0  
4  3.0    5    1    3    2    3    4    3    3  4316715.0  

[5 rows x 43 columns]

In [204]:
# Data fields
# Id : Restaurant id. 
# Open Date : opening date for a restaurant
# City : City that the restaurant is in. Note that there are unicode in the names. 
# City Group: Type of the city. Big cities, or Other. 
# Type: Type of the restaurant. FC: Food Court, IL: Inline, DT: Drive Thru, MB: Mobile
# P1, P2 - P37: There are three categories of these obfuscated data. Demographic data are gathered from third party providers with GIS systems. 
#    These include population in any given area, age and gender distribution, development scales. Real estate data mainly relate to the m2 of the location, 
#    front facade of the location, car park availability. Commercial data mainly include the existence of points of interest including schools, banks, 
#    other QSR operators.
# Revenue: The revenue column indicates a (transformed) revenue of the restaurant in a given year and is the target of predictive analysis. 
#    Please note that the values are transformed so they don't mean real dollar values.

traindata.describe(include='all')

Id   Open Date      City  City Group Type          P1  \
count   137.000000         137       137         137  137  137.000000   
unique         NaN         134        34           2    3         NaN   
top            NaN  02/02/2012  İstanbul  Big Cities   FC         NaN   
freq           NaN           2        50          78   76         NaN   
mean     68.000000         NaN       NaN         NaN  NaN    4.014599   
std      39.692569         NaN       NaN         NaN  NaN    2.910391   
min       0.000000         NaN       NaN         NaN  NaN    1.000000   
25%      34.000000         NaN       NaN         NaN  NaN    2.000000   
50%      68.000000         NaN       NaN         NaN  NaN    3.000000   
75%     102.000000         NaN       NaN         NaN  NaN    4.000000   
max     136.000000         NaN       NaN         NaN  NaN   12.000000   

                P2          P3          P4          P5  ...         P29  \
count   137.000000  137.000000  137.000000  137.000000  ...  137.000000   
unique         NaN         NaN         NaN         NaN  ...         NaN   
top            NaN         NaN         NaN         NaN  ...         NaN   
freq           NaN         NaN         NaN         NaN  ...         NaN   
mean      4.408759    4.317518    4.372263    2.007299  ...    3.135036   
std       1.514900    1.032337    1.016462    1.209620  ...    1.680887   
min       1.000000    0.000000    3.000000    1.000000  ...    0.000000   
25%       4.000000    4.000000    4.000000    1.000000  ...    2.500000   
50%       5.000000    4.000000    4.000000    2.000000  ...    3.000000   
75%       5.000000    5.000000    5.000000    2.000000  ...    3.000000   
max       7.500000    7.500000    7.500000    8.000000  ...    7.500000   

               P30         P31         P32         P33         P34  \
count   137.000000  137.000000  137.000000  137.000000  137.000000   
unique         NaN         NaN         NaN         NaN         NaN   
top            NaN         NaN         NaN         NaN         NaN   
freq           NaN         NaN         NaN         NaN         NaN   
mean      2.729927    1.941606    2.525547    1.138686    2.489051   
std       5.536647    3.512093    5.230117    1.698540    5.165093   
min       0.000000    0.000000    0.000000    0.000000    0.000000   
25%       0.000000    0.000000    0.000000    0.000000    0.000000   
50%       0.000000    0.000000    0.000000    0.000000    0.000000   
75%       4.000000    3.000000    3.000000    2.000000    3.000000   
max      25.000000   15.000000   25.000000    6.000000   24.000000   

               P35         P36         P37       revenue  
count   137.000000  137.000000  137.000000  1.370000e+02  
unique         NaN         NaN         NaN           NaN  
top            NaN         NaN         NaN           NaN  
freq           NaN         NaN         NaN           NaN  
mean      2.029197    2.211679    1.116788  4.453533e+06  
std       3.436272    4.168211    1.790768  2.576072e+06  
min       0.000000    0.000000    0.000000  1.149870e+06  
25%       0.000000    0.000000    0.000000  2.999068e+06  
50%       0.000000    0.000000    0.000000  3.939804e+06  
75%       4.000000    3.000000    2.000000  5.166635e+06  
max      15.000000   20.000000    8.000000  1.969694e+07  

[11 rows x 43 columns]

In [205]:
ds = pd.DataFrame()
#ds['Id'] = traindata.Id
#ds = ds.set_index('Id')

# Open Data
ds['OpenDate'] = traindata['Open Date']
now = datetime.now()
now_date = date(now.year, now.month, now.day)
days_old = []

for i in ds.OpenDate:
    cur_date = i.split('/')
    x = date(int(cur_date[2]), int(cur_date[0]), int(cur_date[1]))
    days_old.append((now_date-x).days)

ds['days_old'] = pd.Series(days_old)

# City
ds['city'] = traindata.City

# One hot encoding for city since all cities are equally important
ds_city = pd.get_dummies(ds.city)
ds = pd.concat([ds, ds_city], axis=1)

del(ds_city)

# One hot encoding for city group
ds_city = pd.get_dummies(ds.city)
ds = pd.concat([ds, ds_city], axis=1)

ds.columns

Index(['OpenDate', 'days_old', 'city', 'Adana', 'Afyonkarahisar', 'Amasya',
       'Ankara', 'Antalya', 'Aydın', 'Balıkesir', 'Bolu', 'Bursa', 'Denizli',
       'Diyarbakır', 'Edirne', 'Elazığ', 'Eskişehir', 'Gaziantep', 'Isparta',
       'Karabük', 'Kastamonu', 'Kayseri', 'Kocaeli', 'Konya', 'Kütahya',
       'Kırklareli', 'Muğla', 'Osmaniye', 'Sakarya', 'Samsun', 'Tekirdağ',
       'Tokat', 'Trabzon', 'Uşak', 'İstanbul', 'İzmir', 'Şanlıurfa', 'Adana',
       'Afyonkarahisar', 'Amasya', 'Ankara', 'Antalya', 'Aydın', 'Balıkesir',
       'Bolu', 'Bursa', 'Denizli', 'Diyarbakır', 'Edirne', 'Elazığ',
       'Eskişehir', 'Gaziantep', 'Isparta', 'Karabük', 'Kastamonu', 'Kayseri',
       'Kocaeli', 'Konya', 'Kütahya', 'Kırklareli', 'Muğla', 'Osmaniye',
       'Sakarya', 'Samsun', 'Tekirdağ', 'Tokat', 'Trabzon', 'Uşak', 'İstanbul',
       'İzmir', 'Şanlıurfa'],
      dtype='object')

In [187]:
ds

OpenDate  days_old        city  Adana  Afyonkarahisar  Amasya  Ankara  \
0    07/17/1999      7376    İstanbul      0               0       0       0   
1    02/14/2008      4242      Ankara      0               0       0       1   
2    03/09/2013      2392  Diyarbakır      0               0       0       0   
3    02/02/2012      2793       Tokat      0               0       0       0   
4    05/09/2009      3792   Gaziantep      0               0       0       0   
..          ...       ...         ...    ...             ...     ...     ...   
132  06/25/2008      4110     Trabzon      0               0       0       0   
133  10/12/2006      4732       İzmir      0               0       0       0   
134  07/08/2006      4828     Kayseri      0               0       0       0   
135  10/29/2010      3254    İstanbul      0               0       0       0   
136  09/01/2009      3677    İstanbul      0               0       0       0   

     Antalya  Aydın  Balıkesir  ...  Osmaniye  Sakarya  Samsun  Tekirdağ  \
0          0      0          0  ...         0        0       0         0   
1          0      0          0  ...         0        0       0         0   
2          0      0          0  ...         0        0       0         0   
3          0      0          0  ...         0        0       0         0   
4          0      0          0  ...         0        0       0         0   
..       ...    ...        ...  ...       ...      ...     ...       ...   
132        0      0          0  ...         0        0       0         0   
133        0      0          0  ...         0        0       0         0   
134        0      0          0  ...         0        0       0         0   
135        0      0          0  ...         0        0       0         0   
136        0      0          0  ...         0        0       0         0   

     Tokat  Trabzon  Uşak  İstanbul  İzmir  Şanlıurfa  
0        0        0     0         1      0          0  
1        0        0     0         0      0          0  
2        0        0     0         0      0          0  
3        1        0     0         0      0          0  
4        0        0     0         0      0          0  
..     ...      ...   ...       ...    ...        ...  
132      0        1     0         0      0          0  
133      0        0     0         0      1          0  
134      0        0     0         0      0          0  
135      0        0     0         1      0          0  
136      0        0     0         1      0          0  

[137 rows x 37 columns]